# DC approximation evaluation
In this notebook we evaluate the DC approximation on bigger environment called `l2RPN_neurips_2020_track1_small`. This concerns power grid use case and aims at providing and reproducing the results provided in the article.

# Environment `l2RPN_neurips_2020_track1_small`  

In [1]:
import pathlib
from pprint import pprint
from lips.benchmark.powergridBenchmark import PowerGridBenchmark
from lips.utils import get_path

In [2]:
# indicate required paths
LIPS_PATH = pathlib.Path().resolve().parent # it is supposed that the notebook had run from getting_started folder
DATA_PATH = LIPS_PATH / "reference_data" / "powergrid" / "l2rpn_neurips_2020_track1_small"
BENCH_CONFIG_PATH = LIPS_PATH / "configurations" / "powergrid" / "benchmarks" / "l2rpn_neurips_2020_track1_small.ini"
SIM_CONFIG_PATH = LIPS_PATH / "configurations" / "powergrid" / "simulators"
BASELINES_PATH = LIPS_PATH / "trained_baselines" / "powergrid"
TRAINED_MODEL_PATH = LIPS_PATH / "trained_models" / "powergrid"
EVALUATION_PATH = LIPS_PATH / "evaluation_results" / "PowerGrid"
LOG_PATH = LIPS_PATH / "lips_logs.log"

## Benchmark1

In [3]:
benchmark1 = PowerGridBenchmark(benchmark_name="Benchmark1",
                                benchmark_path=DATA_PATH,
                                load_data_set=True,
                                log_path=LOG_PATH,
                                config_path=BENCH_CONFIG_PATH
                               )

In [4]:
# to verify the config is loaded appropriately for this benchmark
print("Benchmark name: ", benchmark1.config.section_name)
print("Environment name: ", benchmark1.config.get_option("env_name"))
print("Output attributes: ", benchmark1.config.get_option("attr_y"))
print("Evaluation criteria: ")
pprint(benchmark1.config.get_option("eval_dict"))

Benchmark name:  Benchmark1
Environment name:  l2rpn_neurips_2020_track1_small
Output attributes:  ('a_or', 'a_ex')
Evaluation criteria: 
{'IndRed': ['TIME_INF'],
 'ML': ['MSE_avg', 'MAE_avg', 'mape_avg', 'mape_90_avg', 'TIME_INF'],
 'OOD': ['MSE_avg', 'MAE_avg', 'mape_avg', 'mape_90_avg', 'TIME_INF'],
 'Physics': ['CURRENT_POS']}


In [5]:
# the next few lines are specific for each benchmark and each `AugmentedSimulator`
import grid2op
import warnings
from lips.physical_simulator.dcApproximationAS import DCApproximationAS
with warnings.catch_warnings():
    warnings.filterwarnings("ignore")
    env = grid2op.make(benchmark1.config.get_option("env_name"))
    grid_path = pathlib.Path(env.get_path_env()) / "grid.json"

dc_sim = DCApproximationAS(name="dc_approximation", 
                           benchmark_name="Benchmark1",
                           config_path=BENCH_CONFIG_PATH,
                           grid_path=grid_path)

In [ ]:
EVAL_SAVE_PATH = get_path(EVALUATION_PATH, benchmark1)
dc_metrics_per_dataset_nips_bench1 = benchmark1.evaluate_simulator(augmented_simulator=dc_sim,
                                                                   dataset="all", # other values : "val", "test", "test_ood_topo"
                                                                   save_path=EVAL_SAVE_PATH,
                                                                   save_predictions=True
                                                                  )

### ML-related performances

#### Test data evaluation

In [11]:
print("MAPE90 for a_or: ", dc_metrics_per_dataset_nips_bench1["test"]["ML"]["mape_90_avg"]["a_or"])
print("MAPE90 for a_ex: ", dc_metrics_per_dataset_nips_bench1["test"]["ML"]["mape_90_avg"]["a_ex"])

MAPE90 for a_or:  0.09211532222184272
MAPE90 for a_ex:  0.0935863213806914


#### OOD Generalization evaluation

In [12]:
print("MAPE90 for a_or: ", dc_metrics_per_dataset_nips_bench1["test_ood_topo"]["ML"]["mape_90_avg"]["a_or"])
print("MAPE90 for a_ex: ", dc_metrics_per_dataset_nips_bench1["test_ood_topo"]["ML"]["mape_90_avg"]["a_ex"])

MAPE90 for a_or:  0.0892979240499491
MAPE90 for a_ex:  0.09065118962072517


### Physics compliances

In [13]:
dc_metrics_per_dataset_nips_bench1["test"]["Physics"].keys()

dict_keys(['CURRENT_POS'])

#### Test dataset evaluation

In [14]:
print("1) Current positivity violation:", dc_metrics_per_dataset_nips_bench1["test"]["Physics"]["CURRENT_POS"])#["a_or"]["Violation_proportion"]

1) Current positivity violation: {}


#### OOD Generalization

In [15]:
print("1) Current positivity violation:", dc_metrics_per_dataset_nips_bench1["test_ood_topo"]["Physics"]["CURRENT_POS"])#["a_or"]["Violation_proportion"]

1) Current positivity violation: {}


## Benchmark2

In [7]:
benchmark2 = PowerGridBenchmark(benchmark_name="Benchmark2",
                                benchmark_path=DATA_PATH,
                                load_data_set=True,
                                log_path=LOG_PATH,
                                config_path=BENCH_CONFIG_PATH
                               )

In [8]:
# to verify the config is loaded appropriately for this benchmark
print("Benchmark name: ", benchmark2.config.section_name)
print("Environment name: ", benchmark2.config.get_option("env_name"))
print("Output attributes: ", benchmark2.config.get_option("attr_y"))
print("Evaluation criteria: ")
pprint(benchmark2.config.get_option("eval_dict"))

Benchmark name:  Benchmark2
Environment name:  l2rpn_neurips_2020_track1_small
Output attributes:  ('a_or', 'a_ex', 'p_or', 'p_ex', 'q_or', 'q_ex', 'prod_q', 'load_v', 'v_or', 'v_ex')
Evaluation criteria: 
{'IndRed': ['TIME_INF'],
 'ML': ['MSE_avg', 'MAE_avg', 'mape_avg', 'mape_90_avg', 'TIME_INF'],
 'OOD': ['MSE_avg', 'MAE_avg', 'mape_avg', 'mape_90_avg', 'TIME_INF'],
 'Physics': ['CURRENT_POS',
             'VOLTAGE_POS',
             'LOSS_POS',
             'DISC_LINES',
             'CURRENT_EQ',
             'CHECK_LOSS',
             'CHECK_GC',
             'CHECK_LC',
             'CHECK_VOLTAGE_EQ']}


In [9]:
# the next few lines are specific for each benchmark and each `AugmentedSimulator`
import grid2op
import warnings
from lips.physical_simulator.dcApproximationAS import DCApproximationAS
with warnings.catch_warnings():
    warnings.filterwarnings("ignore")
    env = grid2op.make(benchmark1.config.get_option("env_name"))
    grid_path = pathlib.Path(env.get_path_env()) / "grid.json"

dc_sim = DCApproximationAS(name="dc_approximation", 
                           benchmark_name="Benchmark2",
                           config_path=BENCH_CONFIG_PATH,
                           grid_path=grid_path)

In [ ]:
EVAL_SAVE_PATH = get_path(EVALUATION_PATH, benchmark2)
dc_metrics_per_dataset_nips_bench2 = benchmark2.evaluate_simulator(augmented_simulator=dc_sim,
                                                                   dataset="all", # other values : "val", "test", "test_ood_topo"
                                                                   save_path=EVAL_SAVE_PATH,
                                                                   save_predictions=True
                                                                  )

### ML-related performances

#### Test data evaluation

In [21]:
print("MAPE90 for a_or: ", dc_metrics_per_dataset_nips_bench2["test"]["ML"]["mape_90_avg"]["a_or"])
print("MAPE90 for a_ex: ", dc_metrics_per_dataset_nips_bench2["test"]["ML"]["mape_90_avg"]["a_ex"])
print("MAPE for p_or: ", dc_metrics_per_dataset_nips_bench2["test"]["ML"]["mape_avg"]["p_or"])
print("MAPE for p_ex: ", dc_metrics_per_dataset_nips_bench2["test"]["ML"]["mape_avg"]["p_ex"])
print("MAE for p_or: ", dc_metrics_per_dataset_nips_bench2["test"]["ML"]["MAE_avg"]["p_or"])
print("MAE for p_ex: ", dc_metrics_per_dataset_nips_bench2["test"]["ML"]["MAE_avg"]["p_ex"])
print("MAPE for v_or: ", dc_metrics_per_dataset_nips_bench2["test"]["ML"]["mape_avg"]["v_or"])
print("MAPE for v_ex: ", dc_metrics_per_dataset_nips_bench2["test"]["ML"]["mape_avg"]["v_ex"])
print("MAE for v_or: ", dc_metrics_per_dataset_nips_bench2["test"]["ML"]["MAE_avg"]["v_or"])
print("MAE for v_ex: ", dc_metrics_per_dataset_nips_bench2["test"]["ML"]["MAE_avg"]["v_ex"])

MAPE90 for a_or:  0.08607726908889432
MAPE90 for a_ex:  0.08688336686118878
MAPE for p_or:  0.07510083406666619
MAPE for p_ex:  0.0752763558681588
MAE for p_or:  0.9608471748798383
MAE for p_ex:  1.0005090969300094
MAPE for v_or:  0.025931524108099433
MAPE for v_ex:  0.02667889294542402
MAE for v_or:  4.883819234156204
MAE for v_ex:  4.114566392478297


#### OOD Generalization evaluation

In [22]:
print("MAPE90 for a_or: ", dc_metrics_per_dataset_nips_bench2["test_ood_topo"]["ML"]["mape_90_avg"]["a_or"])
print("MAPE90 for a_ex: ", dc_metrics_per_dataset_nips_bench2["test_ood_topo"]["ML"]["mape_90_avg"]["a_ex"])
print("MAPE for p_or: ", dc_metrics_per_dataset_nips_bench2["test_ood_topo"]["ML"]["mape_avg"]["p_or"])
print("MAPE for p_ex: ", dc_metrics_per_dataset_nips_bench2["test_ood_topo"]["ML"]["mape_avg"]["p_ex"])
print("MAE for p_or: ", dc_metrics_per_dataset_nips_bench2["test_ood_topo"]["ML"]["MAE_avg"]["p_or"])
print("MAE for p_ex: ", dc_metrics_per_dataset_nips_bench2["test_ood_topo"]["ML"]["MAE_avg"]["p_ex"])
print("MAPE for v_or: ", dc_metrics_per_dataset_nips_bench2["test_ood_topo"]["ML"]["mape_avg"]["v_or"])
print("MAPE for v_ex: ", dc_metrics_per_dataset_nips_bench2["test_ood_topo"]["ML"]["mape_avg"]["v_ex"])
print("MAE for v_or: ", dc_metrics_per_dataset_nips_bench2["test_ood_topo"]["ML"]["MAE_avg"]["v_or"])
print("MAE for v_ex: ", dc_metrics_per_dataset_nips_bench2["test_ood_topo"]["ML"]["MAE_avg"]["v_ex"])

MAPE90 for a_or:  0.09784620136324164
MAPE90 for a_ex:  0.0961574901261287
MAPE for p_or:  0.08847901551545259
MAPE for p_ex:  0.08881861816568805
MAE for p_or:  1.2847240290123827
MAE for p_ex:  1.3486002007792932
MAPE for v_or:  0.027389127445560022
MAPE for v_ex:  0.02817785041661958
MAE for v_or:  5.217312458801269
MAE for v_ex:  4.428348349322302


### Physics compliances

In [ ]:
dc_metrics_per_dataset_bench2["test"]["Physics"].keys()

#### Test dataset evaluation

In [18]:
print("1) Current positivity violation:", dc_metrics_per_dataset_nips_bench2["test"]["Physics"]["CURRENT_POS"])#["a_or"]["Violation_proportion"]
print("2) Voltage positivity violation:", dc_metrics_per_dataset_nips_bench2["test"]["Physics"]["VOLTAGE_POS"])
print("3) Loss positivity violation:", dc_metrics_per_dataset_nips_bench2["test"]["Physics"]["LOSS_POS"])
print("4) Disconnected lines violation:", dc_metrics_per_dataset_nips_bench2["test"]["Physics"]["DISC_LINES"])
print("5) Violation of loss to be between [1,4]% of production:", dc_metrics_per_dataset_nips_bench2["test"]["Physics"]["CHECK_LOSS"]["violation_percentage"])
print("6) Violation of global conservation: {}% and its weighted mape: {}".format(dc_metrics_per_dataset_nips_bench2["test"]["Physics"]["CHECK_GC"]["violation_percentage"], dc_metrics_per_dataset_nips_bench2["test"]["Physics"]["CHECK_GC"]["wmape"]))
print("7) Violation of local conservation: {}% and its weighted mape: {}".format(dc_metrics_per_dataset_nips_bench2["test"]["Physics"]["CHECK_LC"]["violation_percentage"], dc_metrics_per_dataset_nips_bench2["test"]["Physics"]["CHECK_LC"]["mape"]))
print("8) Violation proportion of voltage equality at subs:", dc_metrics_per_dataset_nips_bench2["test"]["Physics"]["CHECK_VOLTAGE_EQ"][0][2])

1) Current positivity violation: {}
2) Voltage positivity violation: {}
3) Loss positivity violation: {}
4) Disconnected lines violation: {}
5) Violation of loss to be between [1,4]% of production: 0.0
6) Violation of global conservation: 100.0% and its weighted mape: 1.0000000158419715
7) Violation of local conservation: 2.7777777777777777% and its weighted mape: 0.011069415807126823
8) Violation proportion of voltage equality at subs: 0.020959742699225667


#### OOD Generalization

In [19]:
print("1) Current positivity violation:", dc_metrics_per_dataset_nips_bench2["test_ood_topo"]["Physics"]["CURRENT_POS"])#["a_or"]["Violation_proportion"]
print("2) Voltage positivity violation:", dc_metrics_per_dataset_nips_bench2["test_ood_topo"]["Physics"]["VOLTAGE_POS"])
print("3) Loss positivity violation:", dc_metrics_per_dataset_nips_bench2["test_ood_topo"]["Physics"]["LOSS_POS"])
print("4) Disconnected lines violation:", dc_metrics_per_dataset_nips_bench2["test_ood_topo"]["Physics"]["DISC_LINES"])
print("5) Violation of loss to be between [1,4]% of production:", dc_metrics_per_dataset_nips_bench2["test_ood_topo"]["Physics"]["CHECK_LOSS"]["violation_percentage"])
print("6) Violation of global conservation: {}% and its weighted mape: {}".format(dc_metrics_per_dataset_nips_bench2["test_ood_topo"]["Physics"]["CHECK_GC"]["violation_percentage"], dc_metrics_per_dataset_nips_bench2["test_ood_topo"]["Physics"]["CHECK_GC"]["wmape"]))
print("7) Violation of local conservation: {}% and its weighted mape: {}".format(dc_metrics_per_dataset_nips_bench2["test_ood_topo"]["Physics"]["CHECK_LC"]["violation_percentage"], dc_metrics_per_dataset_nips_bench2["test_ood_topo"]["Physics"]["CHECK_LC"]["mape"]))
print("8) Violation proportion of voltage equality at subs:", dc_metrics_per_dataset_nips_bench2["test_ood_topo"]["Physics"]["CHECK_VOLTAGE_EQ"][0][2])

1) Current positivity violation: {}
2) Voltage positivity violation: {}
3) Loss positivity violation: {}
4) Disconnected lines violation: {}
5) Violation of loss to be between [1,4]% of production: 0.0
6) Violation of global conservation: 100.0% and its weighted mape: 0.9999999921966226
7) Violation of local conservation: 2.7777777777777777% and its weighted mape: 0.015149217172298248
8) Violation proportion of voltage equality at subs: 0.03629023768921451


### Industrial Readiness

In [20]:
print(f'Inference time for test dataset: {dc_metrics_per_dataset_nips_bench2["test"]["IndRed"]["TIME_INF"]:.2f}s')
print(f'Inference time for OOD dataset: {dc_metrics_per_dataset_nips_bench2["test_ood_topo"]["IndRed"]["TIME_INF"]:.2f}s')

Inference time for test dataset: 88.82s
Inference time for OOD dataset: 91.76s
